<a href="https://colab.research.google.com/github/dinky-coder/dinky-coder/blob/main/AI_Agents%26EnvironmentSafety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pattern 1: Reversible Actions

In [2]:
from datetime import datetime

class ReversibleAction:
    def __init__(self, execute_func, reverse_func):
        self.execute = execute_func
        self.reverse = reverse_func
        self.execution_record = None

    def run(self, **args):
        """Execute action and record how to reverse it."""
        result = self.execute(**args)
        self.execution_record = {
            "args": args,
            "result": result,
            "timestamp": datetime.now().isoformat()
        }
        return result

    def undo(self):
        """Reverse the action using recorded information."""
        if not self.execution_record:
            raise ValueError("No action to reverse")
        return self.reverse(**self.execution_record)

# Placeholder functions for demonstration
class MockCalendar:
    def create_event(self, **kwargs):
        print("Creating event with:", kwargs)
        return {"event_id": "mock_event_123"}

    def delete_event(self, event_id):
        print("Deleting event with id:", event_id)

    def send_invite(self, **kwargs):
        print("Sending invite with:", kwargs)
        return {"invite_id": "mock_invite_456"}

    def cancel_invite(self, invite_id):
        print("Canceling invite with id:", invite_id)

calendar = MockCalendar()

# Example using reversible actions
create_event = ReversibleAction(
    execute_func=calendar.create_event,
    reverse_func=lambda **record: calendar.delete_event(record["result"]["event_id"])
)

send_invite = ReversibleAction(
    execute_func=calendar.send_invite,
    reverse_func=lambda **record: calendar.cancel_invite(record["result"]["invite_id"])
)

Pattern 2: Transaction Management

In [3]:
class ActionTransaction:
    def __init__(self):
        self.actions = []
        self.executed = []
        self.committed = False
        self.transaction_id = str(uuid.uuid4())

    def add(self, action: ReversibleAction, **args):
        """Queue an action for execution."""
        if self.committed:
            raise ValueError("Transaction already committed")
        self.actions.append((action, args))

    async def execute(self):
        """Execute all actions in the transaction."""
        try:
            for action, args in self.actions:
                result = action.run(**args)
                self.executed.append(action)
        except Exception as e:
            # If any action fails, reverse everything done so far
            await self.rollback()
            raise e

    async def rollback(self):
        """Reverse all executed actions in reverse order."""
        for action in reversed(self.executed):
            await action.undo()
        self.executed = []

    def commit(self):
        """Mark transaction as committed."""
        self.committed = True

Pattern 3: Staged Execution with Review

In [5]:
import uuid
from typing import List

class StagedActionEnvironment:
    def __init__(self):
        self.staged_transactions = {}
        self.llm = None  # High-capability LLM for review

    def stage_actions(self, task_id: str) -> ActionTransaction:
        """Create a new transaction for staging actions."""
        transaction = ActionTransaction()
        self.staged_transactions[task_id] = transaction
        return transaction

    def review_transaction(self, task_id: str) -> bool:
        """Have LLM review staged actions for safety."""
        transaction = self.staged_transactions.get(task_id)
        if not transaction:
            raise ValueError(f"No transaction found for task {task_id}")
        # Create a description of staged actions
        staged_actions = [
            f"Action: {action.__class__.__name__}\nArgs: {args}"
            for action, args in transaction.actions
        ]

        # The safest way to do this would be to send it for human review, but we can also imagine having a more capable AI system review it before the human to minimize the number of reviews that the human has to do. The more capable AI can review and reject potentially problematic actions earlier.

        review_prompt = f"""Review these staged actions for safety:

        Task ID: {task_id}

        Staged Actions:
        {staged_actions}

        Consider:
        1. Are all actions necessary for the task?
        2. Could any action have unintended consequences?
        3. Are the actions in a safe order?
        4. Is there a safer way to achieve the same goal?

        Should these actions be approved?
        """

        response = self.llm.generate(review_prompt)

        # If approved, notify the human and ask if
        # they want to proceed
        return "approved" in response.lower()
# Example usage:
async def schedule_team_meeting(env: StagedActionEnvironment,
                              attendees: List[str],
                              duration: int):
    """Schedule a team meeting with safety checks."""
    task_id = str(uuid.uuid4())
    transaction = env.stage_actions(task_id)

    # Check availability (execute immediately)
    # available_slots = calendar.check_availability(attendees, duration)
    # if not available_slots:
    #     return {"error": "No available time slots"}

    # best_slot = available_slots[0]

    # Stage the event creation
    transaction.add(create_event,
                   title="Team Meeting",
                   time="some_time", # Placeholder
                   duration=duration)

    # Draft email (execute immediately)
    # email_draft = email.draft_message(
    #     to=attendees,
    #     subject="Team Meeting",
    #     body=f"Team meeting scheduled for {best_slot}"
    # )

    # Stage the email send
    # transaction.add(send_email,
    #                draft_id=email_draft.id)

    # Review staged actions...send to human review
    # or more capable AI for initial filtering
    if env.review_transaction(task_id):
        await transaction.execute()
        transaction.commit()
        return {"status": "scheduled"}
    else:
        return {"status": "rejected"}

Pattern 4: Single Safe Tool vs Multiple Risky Tools

In [7]:
# Placeholder decorator
def register_tool(description):
    def decorator(func):
        func.description = description
        return func
    return decorator

# Placeholder for ActionContext
class ActionContext:
    pass

# Placeholder for email object
class MockEmail:
    def send(self, to, subject, body):
        print(f"Sending email to {to} with subject {subject}")
        return {"status": "sent"}

email = MockEmail()

# Placeholder for calendar object (already defined in a previous cell, but redefined here for clarity in this example)
class MockCalendar:
    def create_event(self, **kwargs):
        print("Creating event with:", kwargs)
        return type("Event", (object,), {"id": "mock_event_123"})()

    def update_event(self, event_id, updates):
        print(f"Updating event {event_id} with {updates}")
        return {"status": "updated"}

calendar = MockCalendar()

# Placeholder for attendee validation
def validate_attendees(attendees):
    print(f"Validating attendees: {attendees}")
    return attendees # Assume all attendees are valid for this example

# Placeholder for finding available times
def find_available_times(attendees, duration, timeframe):
    print(f"Finding available times for {attendees} for {duration} minutes in {timeframe}")
    # Return a dummy available time for the example
    return ["2023-10-27T10:00:00Z"]

# Placeholder for notifications
class MockNotifications:
    def send_meeting_scheduled(self, event_id, attendees):
        print(f"Sending meeting scheduled notification for event {event_id} to {attendees}")

notifications = MockNotifications()


# Approach 1: Multiple loosely constrained tools
@register_tool(description="Create a calendar event")
def create_calendar_event(action_context: ActionContext,
                         title: str,
                         time: str,
                         attendees: List[str]) -> dict:
    """Create a calendar event."""
    return calendar.create_event(title=title,
                               time=time,
                               attendees=attendees)

@register_tool(description="Send email to attendees")
def send_email(action_context: ActionContext,
               to: List[str],
               subject: str,
               body: str) -> dict:
    """Send an email."""
    return email.send(to=to, subject=subject, body=body)

@register_tool(description="Update calendar event")
def update_event(action_context: ActionContext,
                 event_id: str,
                 updates: dict) -> dict:
    """Update any aspect of a calendar event."""
    return calendar.update_event(event_id, updates)

# Approach 2: Single comprehensive safe tool
@register_tool(description="Safely schedule a team meeting")
def schedule_team_meeting(action_context: ActionContext,
                         title: str,
                         description: str,
                         attendees: List[str],
                         duration_minutes: int,
                         timeframe: str = "next_week") -> dict:
    """
    Safely schedule a team meeting with all necessary coordination.

    This tool:
    1. Verifies all attendees are valid
    2. Checks calendar availability
    3. Creates the event at the best available time
    4. Sends appropriate notifications
    5. Handles all error cases
    """

    # Input validation
    if not 15 <= duration_minutes <= 120:
        raise ValueError("Meeting duration must be between 15 and 120 minutes")

    if len(attendees) > 10:
        raise ValueError("Cannot schedule meetings with more than 10 attendees")

    # Verify attendees
    valid_attendees = validate_attendees(attendees)
    if len(valid_attendees) != len(attendees):
        raise ValueError("Some attendees are invalid")

    # Find available times
    available_slots = find_available_times(
        attendees=valid_attendees,
        duration=duration_minutes,
        timeframe=timeframe
    )

    if not available_slots:
        return {
            "status": "no_availability",
            "message": "No suitable time slots found"
        }

    # Create event at best time
    event = calendar.create_event(
        title=title,
        description=description,
        time=available_slots[0],
        duration=duration_minutes,
        attendees=valid_attendees
    )

    # Send notifications
    notifications.send_meeting_scheduled(
        event_id=event.id,
        attendees=valid_attendees
    )

    return {
        "status": "scheduled",
        "event_id": event.id,
        "scheduled_time": available_slots[0]
    }